In [5]:
from operator import itemgetter
from datetime import timedelta
import datetime

from itertools import groupby
from icalendar import Calendar

import ipywidgets as widgets

parcours_dict = {

"G101": "BUT1 : code B1BUE11",
"G102": "BUT1 : code B1BUE11",
"G103": "BUT1 : code B1BUE11",
"G104": "BUT1 : code B1BUE11",
"G105": "BUT1 : code B1BUE11",
"G106": "BUT1 : code B1BUE11",
"G107": "BUT1 : code B1BUE11",
"G108": "BUT1 : code B1BUE11",
    
    
"G101A": "BUT1 : code B1BUE11",
"G102A": "BUT1 : code B1BUE11",
"G103A": "BUT1 : code B1BUE11",
"G104A": "BUT1 : code B1BUE11",
"G105A": "BUT1 : code B1BUE11",
"G106A": "BUT1 : code B1BUE11",
"G107A": "BUT1 : code B1BUE11",
"G108A": "BUT1 : code B1BUE11",
    
    
"G101B": "BUT1 : code B1BUE11",
"G102B": "BUT1 : code B1BUE11",
"G103B": "BUT1 : code B1BUE11",
"G104B": "BUT1 : code B1BUE11",
"G105B": "BUT1 : code B1BUE11",
"G106B": "BUT1 : code B1BUE11",
"G107B": "BUT1 : code B1BUE11",
"G108B": "BUT1 : code B1BUE11",
    
"Gr101": "BUT1 : code B1BUE11",
"Gr102": "BUT1 : code B1BUE11",
"Gr103": "BUT1 : code B1BUE11",
"Gr104": "BUT1 : code B1BUE11",
"Gr105": "BUT1 : code B1BUE11",
"Gr106": "BUT1 : code B1BUE11",
"Gr107": "BUT1 : code B1BUE11",
"Gr108": "BUT1 : code B1BUE11",
    
    
"G302": "BUT2 AII B1BUE312",
"G303": "BUT2 AII B1BUE312",
"G304": "BUT2 AII B1BUE312",
"G305": "BUT2 EME B1BUE311",
"G306": "BUT2 EME B1BUE311",
"G307": "BUT2 ESE B1BUE313",
"G308": "BUT2 ESE B1BUE313",
    
"G302A": "BUT2 AII B1BUE312",
"G303A": "BUT2 AII B1BUE312",
"G304A": "BUT2 AII B1BUE312",
"G305A": "BUT2 EME B1BUE311",
"G306A": "BUT2 EME B1BUE311",
"G307A": "BUT2 ESE B1BUE313",
"G308A": "BUT2 ESE B1BUE313",
    
"G302B": "BUT2 AII B1BUE312",
"G303B": "BUT2 AII B1BUE312",
"G304B": "BUT2 AII B1BUE312",
"G305B": "BUT2 EME B1BUE311",
"G306B": "BUT2 EME B1BUE311",
"G307B": "BUT2 ESE B1BUE313",
"G308B": "BUT2 ESE B1BUE313",
   
"Gr302": "BUT2 AII B1BUE312",
"Gr303": "BUT2 AII B1BUE312",
"Gr304": "BUT2 AII B1BUE312",
"Gr305": "BUT2 EME B1BUE311",
"Gr306": "BUT2 EME B1BUE311",
"Gr307": "BUT2 ESE B1BUE313",
"Gr308": "BUT2 ESE B1BUE313",
"Gr306-Gr308": "BUT2 ESE B1BUE313",            
"Gr503": "BUT3 AII code : B1BUE512",                 
"Gr508": "BUT3 ESE code: B1BUE513",
"Groupe TD EME": "BUT3 EME code: B1BUE511"  
    
 }

#parcours = ['AII', 'EME', 'ESE']

def validate_date(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        return False

def calculate_time(event):
    start = event['DTSTART'].dt
    end = event['DTEND'].dt
    return end - start

def lecturize(event):
    summary = str(event['SUMMARY'])
    return summary

def time_per_lecture_old(events):
    lecture_name = itemgetter(0)
    sorted_events = sorted(events, key=lecture_name)

    for key, group in groupby(sorted_events, lecture_name):
        #print(sum(map(itemgetter(1), group), timedelta()))
        yield (tuple(key), sum(map(itemgetter(1), group), timedelta()))

def parcourize(event, parcours_dict):
    """associate a parcours (AII, EME or ESE) to each lecture"""
    DESCRIPTION = str(event['DESCRIPTION'])
    group = DESCRIPTION.split("\n")[2]
    test = group in parcours_dict.keys()

    if group in parcours_dict.keys():
        track = parcours_dict[group]
        
    #elif [k+"A" for k in parcours_dict.keys()] or [k+"B" for k in parcours_dict.keys()]:
     #   print(group[:-1])
      #  track = parcours_dict[group[:-1]]
    else:        
        track = "! check code ! " + group
    return track

def time_per_lecture(events):
    sorted_events = sorted(events, key=itemgetter(2, 0))
    for (lecture_name, parcours), batch in groupby(sorted_events, key=itemgetter(0, 2)):
        yield (parcours, lecture_name, sum(map(itemgetter(1), batch), timedelta()) )

        
def naive_event_datetime(ev, date_format_str):
    event_datetime = datetime.datetime.strptime(ev['DTSTART'].dt.strftime(date_format_str), date_format_str)
    return event_datetime


uploader = widgets.FileUpload()
display(uploader)

out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout("Cliquer sur le bouton et sélectionner votre fichier .ics extrait à partir d'ADE."+"\n")

@out.capture(clear_output=True)
def run(change):

    for mybunch in uploader.value:
        mybunch_decoded = bytes(uploader.value[mybunch]["content"])
    
    cal = Calendar.from_ical(mybunch_decoded)

    # create the list of tuples (lecture_event, time, parcours) from the calendar
    sessions = [(lecturize(e), calculate_time(e), parcourize(e, parcours_dict)) for e in cal.walk('vevent')]
    
    # Create a generator to group the amount of time per lecture type
    used_time = time_per_lecture(sessions)

    # Sum the entire amount of time including all lectures
    total_time = 0
    for el in used_time:
        
        track = el[0]
        lecture = el[1]
        duration = el[2]
        total_time += duration.total_seconds() / 3600
        out.append_stdout('parcours {} module {} : \t{}h'.format(track, lecture, duration.total_seconds() / 3600)+"\n")

    out.append_stdout('============='+"\n")
    #out.append_stdout('TOTAL\t{}h'.format(total_time)+"\n")
    
    
uploader.observe(run, names="value")



FileUpload(value={}, description='Upload')

In [6]:
out

Output(layout=Layout(border='1px solid black'))

In [16]:
print("")

dict_keys(['ADECal.ics'])

TypeError: string indices must be integers

ADECal.ics


TypeError: string indices must be integers